## Simple example

The `EPyT` python class, epanet, includes properties of the input network model, static properties, public functions, and local functions that directly call the `EPANET` shared object library or the executable. The full API with detailed examples is provided in the link. In the following paragraphs, we introduce a selection of the capabilities of the tool and an illustrative smart water research example.

Through `EPyT`, users can easily load EPANET network models, as demonstrated by importing the module and loading the `Net1.inp` network in a provided example.

### Initialize EPANET Python Toolkit (EPyT)

You should always begin with this command to import the toolkit.

[EPyT](https://github.com/OpenWaterAnalytics/EPyT) is available on [PyPI](https://pypi.org/project/epyt/) and can be installed via `pip install epyt`. To upgrade to the latest version if it's already installed, use `pip install --upgrade epyt`.

In [ ]:
%pip install epyt

In [ ]:
from epyt import epanet

In [ ]:
# Load EPANET Input File
G = epanet('Net1.inp')

In this example, `G` is an epanet object which can be defined mathematically as the set comprised of the network graph topology (such as nodes and links), structural parameters (such as pipe lengths, diameters, etc.), and functions (such as hydraulic solvers, etc.). `G` can also be shared between different functions as an argument.

When the object is constructed, the module reads the `EPANET` input network file and populates more than 500 object parameters. To view some of the parameters and receive assistance with respect to the methods, the user can use the following commands:

In [ ]:
# Lists all available functions and properties
dir(G)

In [ ]:
# Retrieve some examples for the function
help(G.getNodeElevations)

Using object `G`, the user can call all the public toolkit functions. The toolkit contains a large set of functions that allow the user to retrieve or update the network data and simulate hydraulic and quality analysis for different scenarios using the EPANET libraries. Examples of how to retrieve some common network parameter values are provided below:

In [ ]:
# Retrieve Link diameters
diameters = G.getLinkDiameter()
print(diameters)

In [ ]:
# Retrieve Node elevations
elevations = G.getNodeElevations()
print(elevations)

Variables diameters and elevations are two arrays, corresponding to the number of links, and the number of nodes in the `Net1` network, respectively. Note that, in case the network model changes, these parameters will be updated. This is demonstrated in the following example:

In [ ]:
# Link diameter for links 2 & 10
diameters = G.getLinkDiameter([2, 10])
print(diameters)

To modify some parameters, we can use the set commands.

In [ ]:
# Update the link 10 diameter from 100 to 90
G.setLinkDiameter(10, 90)
# Retrieve the diameter of link 10
n_diameter = G.getLinkDiameter(10)
print(n_diameter)

Next, we show how to simulate the water distribution network, such as flows/pressures and water quality. Various functions have been included in the toolkit to simplify the workflows of solving and retrieving the data from the library memory. One way is to solve the hydraulics and quality equations using the `EPANET` library and store the results in data structures.

In [ ]:
# Solve hydraulics in library
# H = G.getComputedHydraulicTimeSeries()

# Solve quality dynamics in library
# Q = G.getComputedQualityTimeSeries()

# Solve all dynamics in library, create a binary file to store the computed values
R = G.getComputedTimeSeries()

To access the different values, the user can use the dot notation, such as `R.Flow` to create an array with the flows and `R.NodeQuality` for the water quality at the nodes. It is important to note that the time intervals may be different for the hydraulic and quality time series, due to the simulation settings; the user can call `R.Time`, to retrieve the time in seconds.

Executing the functions `G.plot()` or `G.plot_ts()`, a figure of the network along with its components is displayed. The nodes i.e., junctions, reservoirs, tanks, and the links, i.e., pipes, valves, and pumps, are depicted using different colors and shapes. An example is provided below:

In [ ]:
# Plot link flows and quality
hrs_time = R.Time / 3600
link_indices = [1, 3, 5, 10]
link_names = G.getLinkNameID(link_indices)
G.plot_ts(X=hrs_time, Y=R.Flow[:, link_indices],
          title=f'Flow, Link IDs: {link_names}', figure_size=[4, 3], legend_location='best',
          xlabel='Time (hrs)', ylabel=f'Flow ({G.units.LinkFlowUnits})',
          marker=None, labels=link_names, save_fig=True) #, filename='figures/paper_flows')
G.plot_ts(X=hrs_time, Y=R.LinkQuality[:, link_indices],
          title=f'Quality, Link IDs: {link_names}', legend_location='best',
          xlabel='Time (hrs)', ylabel=f'Quality', figure_size=[4, 3],
          marker=None, labels=link_names, save_fig=True) #, filename='figures/paper_link_quality')

# Plot node pressures and quality
node_indices = [2, 4, 6, 10]
node_names = G.getNodeNameID(node_indices)
G.plot_ts(X=hrs_time, Y=R.Pressure[:, node_indices], legend_location='best',
          title=f'Pressure, Node IDs: {node_names}', figure_size=[4, 3],
          xlabel='Time (hrs)', ylabel=f'Pressure ({G.units.NodePressureUnits})',
          marker=None, labels=node_names, save_fig=True) #, filename='figures/paper_pressures')

G.plot_ts(X=hrs_time, Y=R.NodeQuality[:, node_indices],
          title=f'Quality, Node IDs: {node_names}', legend_location='best',
          xlabel='Time (hrs)', ylabel=f'Quality', figure_size=[4, 3],
          marker=None, labels=node_names, save_fig=True) #, filename='figures/paper_node_quality')


#### Unload library

In [ ]:
G.unload()